In [2]:
import os
import re
import sys
import importlib.util

# Mapping of import names to pip package names
package_mapping = {
    'paho': 'paho-mqtt',
    'serial': 'pyserial'
}

def find_python_files(base_directory):
    python_files = []
    for root, _, files in os.walk(base_directory):
        for file in files:
            if file.endswith('.py'):
                python_files.append(os.path.join(root, file))
    return python_files

def parse_imports_from_python_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    imports = re.findall(r'^\s*(?:import|from)\s+([a-zA-Z_][\w\.]*)', content, re.MULTILINE)
    return imports

def get_top_level_modules(imports):
    top_level_modules = set()
    for imp in imports:
        top_module = imp.split('.')[0]
        top_level_modules.add(top_module)
    return top_level_modules

def is_standard_lib(module_name):
    spec = importlib.util.find_spec(module_name)
    if spec is None:
        return False
    if 'site-packages' in spec.origin or 'dist-packages' in spec.origin:
        return False
    return True

def filter_third_party_modules(modules):
    third_party_modules = set()
    for module in modules:
        if not is_standard_lib(module):
            third_party_modules.add(module)
    return third_party_modules

def map_to_pip_package(modules):
    pip_packages = set()
    for module in modules:
        pip_package = package_mapping.get(module, module)
        pip_packages.add(pip_package)
    return pip_packages

def write_requirements_file(base_directory, requirements):
    requirements_file_path = os.path.join(base_directory, 'requirements.txt')
    with open(requirements_file_path, 'w') as file:
        for req in sorted(requirements):
            file.write(req + '\n')

def main(base_directory):
    all_imports = set()

    # Collect requirements from import statements in Python files
    python_files = find_python_files(base_directory)
    for py_file in python_files:
        imports = parse_imports_from_python_file(py_file)
        all_imports.update(imports)

    # Get top-level modules to avoid listing submodules
    top_level_modules = get_top_level_modules(all_imports)

    # Filter out standard library modules
    third_party_modules = filter_third_party_modules(top_level_modules)

    # Map import names to pip package names
    pip_packages = map_to_pip_package(third_party_modules)

    # Write all unique third-party requirements to a requirements.txt file
    write_requirements_file(base_directory, pip_packages)
    print(f"requirements.txt file has been created in {base_directory}")

if __name__ == "__main__":
    base_directory = "/home/vincent/MySSD/JupyterProjects/AAA_projects/UnlimitedResearchCooperative/Synthetic_Intelligence_Labs/Bio-Silicon-Synergetic-Intelligence-System/Software/system/shuffleboard_1D/script_based_system/digital_or_analogue_system"
    main(base_directory)


requirements.txt file has been created in /home/vincent/MySSD/JupyterProjects/AAA_projects/UnlimitedResearchCooperative/Synthetic_Intelligence_Labs/Bio-Silicon-Synergetic-Intelligence-System/Software/system/shuffleboard_1D/script_based_system/digital_or_analogue_system
